# Configuration

In [1]:
input_file = 'hdfs:///files/salmonella/SRR15404285.fasta'
output_file = 'hdfs:///files/salmonella/SRR15404285.pickleb64.320'
partitions = 320

In [2]:
import pickle
import base64

import findspark
findspark.init()
findspark.find()

'/usr/local/spark/python/pyspark'

In [3]:
from pyspark.sql import SparkSession

spark = (SparkSession
         .builder
         .master("yarn")
         .appName("python-testing")
         .config("spark.executor.instances", 8)
         .config("spark.executor.memory", "800m")
         .getOrCreate())
sc = spark.sparkContext
sc

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-04-23 17:28:49,992 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2022-04-23 17:28:57,894 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


<SparkContext master=yarn appName=python-testing>

In [4]:
reads = sc.textFile(input_file).filter(lambda x: not x.startswith('>')).zipWithIndex().map(lambda x: (x[1], x[0]))
reads.take(1)

[(0,
  'TGCCGNCCTGAGCGAAAGCCTGCTGGAAGAAGTAGCTTCGCTGGTGGAATGGCCGGTGGTATTGACGGCGAAATT')]

In [5]:
pickled = reads.map(lambda x: pickle.dumps(x))
pickled.take(1)

[b'\x80\x04\x95S\x00\x00\x00\x00\x00\x00\x00K\x00\x8cKTGCCGNCCTGAGCGAAAGCCTGCTGGAAGAAGTAGCTTCGCTGGTGGAATGGCCGGTGGTATTGACGGCGAAATT\x94\x86\x94.']

In [6]:
base64d = pickled.map(lambda x: base64.b64encode(x))
base64d.take(1)

[b'gASVUwAAAAAAAABLAIxLVEdDQ0dOQ0NUR0FHQ0dBQUFHQ0NUR0NUR0dBQUdBQUdUQUdDVFRDR0NUR0dUR0dBQVRHR0NDR0dUR0dUQVRUR0FDR0dDR0FBQVRUlIaULg==']

In [7]:
base64d.repartition(partitions).saveAsTextFile(output_file)

In [4]:
[x for x in range(3)]

[0, 1, 2]